In [1]:
import os
import pandas as pd 
import numpy as np 

# visualisation
import matplotlib.pylab as plt
import matplotlib.font_manager as fm

# our scripts 
import src.processing as sp 
import settings as st
import utils.helpers as hp
import utils.imaging as ui

### Correct for the locations of the images 

In [2]:
# dr5_desc = sp.correct_location('gz_decals_volunteers_5', save = True, filename='dr5_votes')

### The data with the votes

In [3]:
dr5 = hp.read_parquet(st.data_dir, 'descriptions/dr5_votes')

### Generate the labels 

In [4]:
labels = sp.generate_labels(dr5, save=False)

In [5]:
train_val_test = sp.split_data(labels, 0.2, 0.2, save=False)

In [6]:
train_val_test['train'].shape

(151970, 36)

In [7]:
train_val_test['validate'].shape

(50658, 36)

In [8]:
train_val_test['test'].shape

(50658, 36)

In [10]:
labels.shape

(253286, 36)